In [ ]:
!pip install citipy

In [ ]:
import matplotlib.pyplot as plt
import requests
import pandas as pd
from config import api_key
from citipy import citipy

In [ ]:
# Create base url
base_url = f"http://api.openweathermap.org/data/2.5/weather?appid={api_key}"
# define units
units = "imperial"

# Build query url with units
query_url = f"{base_url}&units={units}&q="

In [ ]:
cities = []
temperature = []
humidity = []
cloudiness = []
wind_speed = []

In [ ]:
for _ in range(0,501):
    
    # Find city and append it to a list
    city_name = citipy.nearest_city(longitude, latitude)
    cities.append(city_name)
    
    # Create url for city
    full_url = f"{query_url}{city_name}"
    # Find json response for city
    response = requests.get(full_url).json()
    
    temp = 
    humid = 
    cloud = 
    wind = 
    
    temperature.append()
    humidity.append()
    cloudiness.append()
    wind_speed.append()
    
    
    